In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import NASNetMobile
from keras.applications.nasnet import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 49673580
EPOCHS = 50
size = 64
batchsize = 512
lw = 6
channel = 1
STEPS = TOT / EPOCHS / batchsize / 3
NCATS = 340

In [3]:
K.clear_session()
# base_model = MobileNet(input_shape=(size, size, 3), alpha=1., weights='imagenet', include_top=False)
# x = base_model.output
# # x = GlobalAveragePooling2D()(x)
# x = Flatten()(x)
# x = Dropout(0.5)(x)

# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# x = Dense(64)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# predictions = Dense(NCATS, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

model = NASNetMobile(input_shape=(size, size, 1), weights=None, classes=NCATS)

model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 31, 31, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 31, 31, 32)   128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 31, 31, 32)   0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

Total params: 4,628,520
Trainable params: 4,591,782
Non-trainable params: 36,738
__________________________________________________________________________________________________
None


In [4]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, channel=channel)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [5]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 64, 64, 1) (34000, 340)
Validation array memory 0.52 GB


In [6]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel)

In [7]:
model_prefix = 'nasnetmobile_raw_head_{}_lw{}'.format(size, lw)
fold = 9
callbks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    callbacks.ModelCheckpoint("./models/{}.model".format(model_prefix),
                                monitor='val_categorical_accuracy', 
                                mode = 'max', save_best_only=True, verbose=1),
    callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
]

In [8]:
hists = []
hist = model.fit_generator(  
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/50
647/646 [==============================] - 358s 553ms/step - loss: 3.3505 - categorical_crossentropy: 3.3505 - categorical_accuracy: 0.2720 - top_3_accuracy: 0.4498 - val_loss: 16.0707 - val_categorical_crossentropy: 16.0707 - val_categorical_accuracy: 0.0029 - val_top_3_accuracy: 1.0000

Epoch 00001: val_categorical_accuracy improved from -inf to 0.00294, saving model to ./models/nasnetmobile_raw_head_64_lw6.model
Epoch 2/50
647/646 [==============================] - 322s 498ms/step - loss: 2.2935 - categorical_crossentropy: 2.2935 - categorical_accuracy: 0.4590 - top_3_accuracy: 0.6709 - val_loss: 16.0707 - val_categorical_crossentropy: 16.0707 - val_categorical_accuracy: 0.0029 - val_top_3_accuracy: 1.0000

Epoch 00002: val_categorical_accuracy did not improve from 0.00294
Epoch 3/50
647/646 [==============================] - 322s 497ms/step - loss: 1.9437 - categorical_crossentropy: 1.9437 - categorical_accuracy: 0.5294 - top_3_accuracy: 0.7371 - val_loss: 15.8627 - val_

KeyboardInterrupt: 

In [ ]:
model.load_weights('./models/{}.model'.format(model_prefix))

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head() 
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

In [ ]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

In [ ]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

In [ ]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

In [ ]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}.csv'.format(model_prefix), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)